## Data Pre-processing:

In [380]:
#Importing libraries:
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ankush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ankush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [381]:
df = pd.read_csv("techcrunch_posts.csv")
df.head(5)

,authors,category,content,date,id,img_src,section,tags,title,topics,url
0,Dale Chang,Startups,\n\nTech investing isn’t what it used to be — ...,2016-10-15,1401293,https://tctechcrunch2011.files.wordpress.com/2...,startups/,data,How startups can use data to grow smarter,NaN,https://techcrunch.com/2016/10/15/how-startups...
1,"Ben Narasin,Jeremy Abelson",Fundings & Exits,\n\nThe IPO market is poised to make an almost...,2016-10-14,1399050,https://tctechcrunch2011.files.wordpress.com/2...,startups/,ipo,"While the IPO market roars back, venture remai...",NaN,https://techcrunch.com/2016/10/14/while-the-ip...
2,Matthew Lynley,Apps,Under pressure from shoppers complaining about...,2016-10-14,1401954,https://tctechcrunch2011.files.wordpress.com/2...,startups/,apoorva-mehta,"Instacart reverses course, re-introducing tips...",instacart,https://techcrunch.com/2016/10/14/instacart-re...
3,"Matthew Lynley,Ingrid Lunden",Fundings & Exits,"PayPal, the giant digital payments company tha...",2016-10-14,1390023,https://tctechcrunch2011.files.wordpress.com/2...,startups/,gofundme,PayPal mulled buying GoFundMe,paypal,https://techcrunch.com/2016/10/14/paypal-gobuyme/
4,Megan Rose Dickey,Startups,"Uber, the ridesharing behemoth that recently b...",2016-10-13,1401633,https://tctechcrunch2011.files.wordpress.com/2...,startups/,lawsuits,Uber hits another roadblock,uber,https://techcrunch.com/2016/10/13/uber-hits-an...


In [382]:
#df.drop(['id','date','img_src','section','topics','url'],axis=1,inplace=True)
df_main = df[['content','title','tags']]
df_main.dropna(inplace=True)

C:\Users\Ankush\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [383]:
df_main['content'] = df_main['content'].str.lstrip()
df_main['title'] = df_main['title'].str.lstrip()
df_main['tags'] = df_main['tags'].str.lstrip()

C:\Users\Ankush\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Ankush\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Ankush\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

In [384]:
def checking_Data(index):
    row = df_main[df_main.index == index][['title', 'content', 'tags']].values[0]
    if len(row) > 0:
        print('Title:',row[0])
        print('Content:',row[1])
        print('Tag:', row[2])
checking_Data(0)

Title: How startups can use data to grow smarter
Content: Tech investing isn’t what it used to be — even compared to six months ago.
Investors are applying greater scrutiny to deals. Many wonder if the days of the mega round that produced the likes of Snapchat, Uber and Magic Leap are fading quickly. Founders are realizing they can’t favor growth over profitability, or vice-versa — both are crucial to success. There’s a reason VCs are calling for “a return to fundamentals” — they don’t want to see newly minted unicorns shrivel into unicorpses.
Companies need to spend their capital carefully, with an eye on steady, sustainable growth. As the pendulum continues to swing more toward smart growth, startups are naturally forced to rethink their approach to operations. Based on my experience advising portfolio companies and consulting, I believe that data-driven operations is the new table stakes for survival today. Here are a few fundamentals of data-driven ops I’d share with any company st

In [385]:
#Cleansing Content and Tags:
replace_blank_regex = re.compile('[/(){}\[\]\|@,;]')
symbols_removel_regex = re.compile('[^0-9a-z #+_]')
stopword_det = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_blank_regex.sub(' ', text) # replace replace_blank_regex symbols by space in text
    text = symbols_removel_regex.sub('', text) # delete symbols which are in symbols_removel_regex from text
    text = ' '.join(word for word in text.split() if word not in stopword_det) # delete stopwors from text
    return text
    
df_main['content'] = df_main['content'].apply(clean_text)
df_main['title'] = df_main['title'].apply(clean_text)
checking_Data(0)

C:\Users\Ankush\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Title: startups use data grow smarter
Content: tech investing isnt used even compared six months agoinvestors applying greater scrutiny deals many wonder days mega round produced likes snapchat uber magic leap fading quickly founders realizing cant favor growth profitability viceversa crucial success theres reason vcs calling return fundamentals dont want see newly minted unicorns shrivel unicorpsescompanies need spend capital carefully eye steady sustainable growth pendulum continues swing toward smart growth startups naturally forced rethink approach operations based experience advising portfolio companies consulting believe datadriven operations new table stakes survival today fundamentals datadriven ops id share company straddling line highgrowth sustainabilityone important decisions choosing set benchmarks metrics actually help measure progress toward specific goals whether profitability growth dont reinvent wheel look companies measured similar stages youre enterprise saas compan

C:\Users\Ankush\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [352]:
#Lemmatizing Content and Title:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    lem_sentence=[]
    lem_sentence= [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

    return " ".join(lem_sentence) 

df_main['content_lemmatized'] = df_main.content.apply(lemmatize_text)
df_main['title_lemmatized'] = df_main.title.apply(lemmatize_text)

C:\Users\Ankush\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Ankush\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [386]:
df_main.head()

,content,title,tags
0,tech investing isnt used even compared six mon...,startups use data grow smarter,data
1,ipo market poised make almost complete 180 tur...,ipo market roars back venture remains leery,ipo
2,pressure shoppers complaining losing ability r...,instacart reverses course reintroducing tips s...,apoorva-mehta
3,paypal giant digital payments company spun eba...,paypal mulled buying gofundme,gofundme
4,uber ridesharing behemoth recently began opera...,uber hits another roadblock,lawsuits
